In [2]:
import os
from numpy import loadtxt
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [3]:
# Join dataset and produce the result dataframe
customer = pd.read_csv("customer.csv",index_col=False)
customer_df = customer[["customerEmail","No_Transactions","No_Orders","No_Payments","Fraud"]]
transaction = pd.read_csv("transaction.csv",index_col=False)
transaction_df = transaction[["customerEmail","paymentMethodType","transactionAmount","orderState","paymentMethodProvider"]]
print(customer_df.head())
print(transaction_df.head())
result = pd.merge(transaction_df,customer_df,on='customerEmail')
print(result.head())




               customerEmail  No_Transactions  No_Orders  No_Payments  Fraud
0     josephhoward@yahoo.com                2          2            1  False
1     evansjeffery@yahoo.com                3          3            7   True
2  andersonwilliam@yahoo.com                5          3            2  False
3      rubenjuarez@yahoo.com                3          3            1  False
4           uchen@malone.com                7          7            6   True
            customerEmail paymentMethodType  transactionAmount orderState  \
0  josephhoward@yahoo.com              card                 18    pending   
1  josephhoward@yahoo.com              card                 26  fulfilled   
2  evansjeffery@yahoo.com           bitcoin                 45  fulfilled   
3  evansjeffery@yahoo.com           bitcoin                 23  fulfilled   
4  evansjeffery@yahoo.com           bitcoin                 43  fulfilled   

  paymentMethodProvider  
0          JCB 16 digit  
1          JCB 16 digit

In [4]:
# Data Preparation - Conversion from categorical values to numeric values and save numeric data frame to training_data_final.csv
arr_customer=result['customerEmail'].unique()
result['cust_id']=result['customerEmail'].apply(lambda x: arr_customer.tolist().index(x))
arr_method=result['paymentMethodType'].unique()
result['pay_method']=result['paymentMethodType'].apply(lambda x: arr_method.tolist().index(x))
arr_method_provider=result['paymentMethodProvider'].unique()
result['pay_method_provider']=result['paymentMethodProvider'].apply(lambda x: arr_method_provider.tolist().index(x))
arr_order_state=result['orderState'].unique()
result['order_state_num']=result['orderState'].apply(lambda x: arr_order_state.tolist().index(x))
arr_fraud=result['Fraud'].unique()
result['fraud_num']=result['Fraud'].apply(lambda x: arr_fraud.tolist().index(x))
num_df = result[["cust_id","order_state_num","pay_method","pay_method_provider","No_Transactions","No_Orders","No_Payments","transactionAmount","fraud_num"]]
num_df.to_csv("training_data_final.csv", sep=',', encoding='utf-8',index=False)
print(num_df.head())

   cust_id  order_state_num  pay_method  pay_method_provider  No_Transactions  \
0        0                0           0                    0                2   
1        0                1           0                    0                2   
2        1                1           1                    1                3   
3        1                1           1                    1                3   
4        1                1           1                    2                3   

   No_Orders  No_Payments  transactionAmount  fraud_num  
0          2            1                 18          0  
1          2            1                 26          0  
2          3            7                 45          1  
3          3            7                 23          1  
4          3            7                 43          1  


In [5]:
dataset = pd.read_csv("training_data_final.csv",index_col=False)
X = dataset.iloc[:,0:8]
Y = dataset.iloc[:,8]
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = XGBClassifier()
model.fit(X_train, y_train)


# Create Prediction Record and Predict if Fraud is True or False
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Accuracy: 97.05%
